# 모바일 적용을 위한 모델 경량화

In [2]:
import torch

from classifier.models.cnn1d import PositionClassifier

In [5]:
weights = "./weights/model.pt"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def load(fn, device):
    d = torch.load(fn, map_location=device)
    
    return d['model'], d['config']

model_dict, train_config = load(weights, device)
print(train_config)

Namespace(weights='./weights/model.pt', gpu_id=-1, train_ratio=0.8, n_epochs=100, batch_size=256, dropout_p=0.3, verbose=1)


In [6]:
model = PositionClassifier(4, 240).to(device)
model.eval()
model.load_state_dict(model_dict)

<All keys matched successfully>

In [8]:
from torch.utils.mobile_optimizer import optimize_for_mobile

example = torch.rand(5,4).unsqueeze(0)
print(example)
traced_script_module = torch.jit.trace(model, example)
optimized_traced_model = optimize_for_mobile(traced_script_module)
optimized_traced_model.save("./weights/for_mobile.pt")

tensor([[[0.6364, 0.2302, 0.5481, 0.4537],
         [0.0256, 0.0420, 0.0015, 0.7533],
         [0.3251, 0.7167, 0.5351, 0.7316],
         [0.4654, 0.0596, 0.5214, 0.0620],
         [0.4713, 0.4396, 0.3236, 0.9658]]])
